# Image Filtering: Spatial Methods

In [ ]:
%matplotlib inline

import numpy as np

import matplotlib.image as img
import matplotlib.pyplot as plt

from skimage import io
from skimage.filters import rank
from skimage.util import img_as_ubyte

from scipy.ndimage import convolve

In [ ]:
def print_imginfo(I):
    print(type(I))
    print(I.shape, I.dtype)
    print('Data range:', np.min(I), 'to', np.max(I))

In [ ]:
I1 = io.imread("../../images/umbc.png", as_gray=True)
I1 = img_as_ubyte(I1) # needed for skimage.filters.rank

io.imshow(I1)

In [ ]:
# add salt-and-pepper noise
rng = np.random.default_rng(472)
noise = rng.random(I1.shape)

In = np.copy(I1)
In[noise > 0.95] = 255
In[noise < 0.05] = 0

io.imshow(In)

## Non-Linear Filter: Kernel Masks (Footprints)

In [ ]:
def box(N, M=None):
    if M == None:
        M = N
        
    h = np.ones((N,M))
    return h

def disk(N):
    radius = (N-1)/2
    cu, cv = (N//2, N//2)

    u, v = np.mgrid[:N, :N]
    dist_from_center = np.sqrt((u-cu)**2+(v-cv)**2)

    h = np.where(dist_from_center <= radius, 1, 0)
    
    return h

## Non-Linear Filter: Alpha-Trimmed Mean

In [ ]:
def mean_filter(I, kernel):
    h = kernel/kernel.sum()
    Ip = convolve(I, h, mode='reflect')
    return Ip

In [ ]:
def alphamean_filter(I, kernel, alpha):
    Ip = np.empty_like(I)
    
    h1 = kernel.shape[0]; h0 = h1 // 2;
    w1 = kernel.shape[1]; w0 = w1 // 2;
    
    Iz = np.pad(I, ((h0,h0),(w0,w0)), mode='reflect')
    
    for u in np.arange(0, I.shape[0]):
        for v in np.arange(0, I.shape[1]):
            values = Iz[u:u+h1, v:v+w1]  # slicing
            values = np.extract(0.0 < kernel, values)
            values = np.sort(values)
            
            M = values.size
            p = np.min([alpha*M, M/2])
            p = np.uint32(np.around(p))
            
            Ip[u][v] = np.mean(values[p:M-p])
            
    return Ip

In [ ]:
%%time
I2 = mean_filter(In, box(5))
io.imshow(I2)

In [ ]:
%%time
I3 = alphamean_filter(In, box(3), 0.45)
io.imshow(I3)

## Non-Linear Filter: Median (Min, Max, Percentile)

In [ ]:
def median_filter(I, kernel):
    Ip = np.empty_like(I)
    
    h1 = kernel.shape[0]; h0 = h1 // 2;
    w1 = kernel.shape[1]; w0 = w1 // 2;
    
    Iz = np.pad(I, ((h0,h0),(w0,w0)), mode='reflect')
    
    for u in np.arange(0, I.shape[0]):
        for v in np.arange(0, I.shape[1]):
            values = Iz[u:u+h1, v:v+w1]
            values = np.extract(0.0 < kernel, values)
            Ip[u][v] = np.median(values)
            
    return Ip

In [ ]:
I2 = median_filter(In, box(3))
io.imshow(I2)

## Non-Linear Filter: Gradient

In [ ]:
def gradient_filter(I, kernel):
    Ip = np.empty_like(I)
    
    h1 = kernel.shape[0]; h0 = h1 // 2;
    w1 = kernel.shape[1]; w0 = w1 // 2;
    
    Iz = np.pad(I, ((h0,h0),(w0,w0)), mode='reflect')
    
    for u in np.arange(0, I.shape[0]):
        for v in np.arange(0, I.shape[1]):
            values = Iz[u:u+h1, v:v+w1]
            value = np.extract(0.0 < kernel, values)
            Ip[u][v] = np.max(values) - np.min(values)
            
    return Ip

In [ ]:
I2 = gradient_filter(I1, box(3))
io.imshow(I2)

## Non-Linear Filter: SKImage 

See //https://scikit-image.org/docs/stable/api/skimage.filters.rank for a list of nonlinear skimage filters. The ones shown below are merely an appetizer.

Arithmetic mean: $\frac{1}{n} \sum_{i=1}^n x_i$

Geometric mean: $\sqrt[n]{x_1 x_2 \cdots x_n}$

In [ ]:
I2 = rank.geometric_mean(I1, disk(3)) # n-root(n-value product)
io.imshow(I2)

Entropy: $0 \leq - \sum_{i=1}^{n} p_i \log p_i \leq \log n$

In [ ]:
I2 = rank.entropy(I1, box(3)) # local entropy
I2 /= np.log2(9)              # normalize to 0:1 range
#I2 = np.where(I2 < 0.75, 0.0, 1.0)  # threshold
io.imshow(I2)

In [ ]:
I2 = rank.majority(I1, disk(3)) # most common local value
io.imshow(I2)

In [ ]:
I2 = rank.enhance_contrast(I1, disk(3)) # use closest local min/max value
io.imshow(I2)